# Lab  1 -Advanced Machine Learning
### Paresh Subedi 19BTRCR065
---
## 1. Build a machine learning model for house price prediction analysis using lasso and ridge regression 

In [1]:
# importing libraries...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# importing dataset...
df = pd.read_csv("housingprices.csv")
df.head()

,Unnamed: 0,Address,Zip,Price,Area,Room,Lon,Lat
0,1,"Blasiusstraat 8 2, Amsterdam",1091 CR,685000.0,64,3,4.907736,52.356157
1,2,"Kromme Leimuidenstraat 13 H, Amsterdam",1059 EL,475000.0,60,3,4.850476,52.348586
2,3,"Zaaiersweg 11 A, Amsterdam",1097 SM,850000.0,109,4,4.944774,52.343782
3,4,"Tenerifestraat 40, Amsterdam",1060 TH,580000.0,128,6,4.789928,52.343712
4,5,"Winterjanpad 21, Amsterdam",1036 KN,720000.0,138,5,4.902503,52.410538


In [3]:
# finding null values...
df.isna().sum()

Unnamed: 0    0
Address       0
Zip           0
Price         4
Area          0
Room          0
Lon           0
Lat           0
dtype: int64

In [4]:
df.shape

(924, 8)

In [5]:
df.nunique()

Unnamed: 0    924
Address       919
Zip           834
Price         226
Area          193
Room           13
Lon           894
Lat           886
dtype: int64

In [6]:
# dropping null values...
df.dropna(inplace=True)
df.isna().sum()

Unnamed: 0    0
Address       0
Zip           0
Price         0
Area          0
Room          0
Lon           0
Lat           0
dtype: int64

In [7]:
df.corr()['Price']

Unnamed: 0   -0.116311
Price         1.000000
Area          0.835090
Room          0.623448
Lon          -0.013561
Lat           0.062196
Name: Price, dtype: float64

In [8]:
df.drop(['Address','Zip','Unnamed: 0'],axis = 1,inplace= True)
df.head()

,Price,Area,Room,Lon,Lat
0,685000.0,64,3,4.907736,52.356157
1,475000.0,60,3,4.850476,52.348586
2,850000.0,109,4,4.944774,52.343782
3,580000.0,128,6,4.789928,52.343712
4,720000.0,138,5,4.902503,52.410538


In [9]:
x= df.drop('Price',axis=1)
y=df['Price']
x.describe()

,Area,Room,Lon,Lat
count,920.000000,920.00000,920.000000,920.000000
mean,95.607609,3.56413,4.888652,52.363271
std,56.849699,1.57103,0.053118,0.024054
min,21.000000,1.00000,4.644819,52.291519
25%,60.000000,3.00000,4.855834,52.351925
50%,83.000000,3.00000,4.886818,52.364499
75%,113.000000,4.00000,4.922337,52.377545
max,623.000000,14.00000,5.029122,52.423805


In [10]:
# splitting dataset to training and testing...
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=50)

In [11]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
model = reg.fit(X_train,y_train)
print("training score ",model.score(X_train,y_train),"\ntesting score ",model.score(X_test,y_test))

training score  0.721879270330792 
testing score  0.6739810261778038


In [12]:
y_pred_lr = model.predict(X_test)

In [13]:
from sklearn.metrics import mean_absolute_error as mae
print("mean absolute error = ",mae(y_test,y_pred_lr))

mean absolute error =  185013.4341818313


In [14]:
from sklearn.linear_model import Lasso
lass_reg = Lasso(random_state=42)
lass_reg.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': 'deprecated',
 'positive': False,
 'precompute': False,
 'random_state': 42,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [15]:
alphas = np.array([500,0.01,0.05,0.1,0.5,1,10,50,100,])
iter = np.array([1000,100,10])

In [16]:
import warnings 
warnings.filterwarnings('ignore')

In [17]:
from sklearn.model_selection import GridSearchCV as gscv
grid = gscv(estimator = lass_reg, param_grid={'alpha' : alphas , 'max_iter':iter},cv = 5)
grid.fit(X_train,y_train)
y_pred_la = grid.predict(X_test)
print('for lasso regularization',grid.best_params_)
print('Training score: ',grid.score(X_train,y_train))
print('Testing score: ',grid.score(X_test,y_test))

for lasso regularization {'alpha': 100.0, 'max_iter': 10}
Training score:  0.7218025661640269
Testing score:  0.6734010769895973


In [18]:
resultdf = pd.DataFrame(grid.cv_results_)
print(resultdf[['param_max_iter','param_alpha','mean_test_score']])

   param_max_iter param_alpha  mean_test_score
0            1000       500.0         0.709562
1             100       500.0         0.709562
2              10       500.0         0.709734
3            1000        0.01         0.709899
4             100        0.01         0.709899
5              10        0.01         0.710078
6            1000        0.05         0.709899
7             100        0.05         0.709899
8              10        0.05         0.710078
9            1000         0.1         0.709899
10            100         0.1         0.709899
11             10         0.1         0.710078
12           1000         0.5         0.709900
13            100         0.5         0.709900
14             10         0.5         0.710079
15           1000         1.0         0.709900
16            100         1.0         0.709900
17             10         1.0         0.710079
18           1000        10.0         0.709912
19            100        10.0         0.709912
20           

In [19]:
from sklearn.linear_model import Ridge
ridg_reg = Ridge()
ridg_reg.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [20]:
grid = gscv(estimator = ridg_reg, param_grid={'alpha' : alphas , 'max_iter':iter},cv = 5)
grid.fit(X_train,y_train)
y_pred_ri = grid.predict(X_test)
print('For ridge regularization\n',grid.best_params_)
print(grid.best_score_)

For ridge regularization
 {'alpha': 500.0, 'max_iter': 1000}
0.7108733095911026


In [21]:
models = {
    'lasso' : {
        'model' : lass_reg,
        'params' : {
            'alpha' : alphas , 'max_iter':iter
        }
    } ,
    'ridge' : {
        'model' : ridg_reg,
        'params' : {
            'alpha' : alphas , 'max_iter': iter
        }
    }
}


scores = []


for modelname, mp in models.items():
    grid = gscv(mp['model'],mp['params'],cv = 5)
    grid.fit(X_train,y_train)
    scores.append(
        {
            'model' : modelname,
            'best_score' : grid.best_score_,
            'best_parameters' : grid.best_params_
        }
    )
    
scores

[{'model': 'lasso',
  'best_score': 0.7101364501189597,
  'best_parameters': {'alpha': 100.0, 'max_iter': 10}},
 {'model': 'ridge',
  'best_score': 0.7108733095911026,
  'best_parameters': {'alpha': 500.0, 'max_iter': 1000}}]

## 3. Compare the differences between the accuracies obtained using ridge and lasso regression in first experiment.

In [22]:
finaldf = pd.DataFrame(scores, columns=['model','best_score','best_parameters'])
finaldf

,model,best_score,best_parameters
0,lasso,0.710136,"{'alpha': 100.0, 'max_iter': 10}"
1,ridge,0.710873,"{'alpha': 500.0, 'max_iter': 1000}"


## 4. For the above build regression model, perform model evaluation, feature selection and parameter tuning.

In [23]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

print("For linear regression: \nr^2 value is ",
      r2_score(y_test,y_pred_lr),
      "\nmean absolute percentage error: ",
      mean_absolute_percentage_error(y_test,y_pred_lr))
print("For lasso regression: \nr^2 value is ",
      r2_score(y_test,y_pred_la),
      "\nmean absolute percentage error: ",
      mean_absolute_percentage_error(y_test,y_pred_la))
print("For ridge regression: \nr^2 value is ",
      r2_score(y_test,y_pred_ri),
      "\nmean absolute percentage error: ",
      mean_absolute_percentage_error(y_test,y_pred_ri))

For linear regression: 
r^2 value is  0.6739810261778038 
mean absolute percentage error:  0.31228256387448644
For lasso regression: 
r^2 value is  0.6734010769895973 
mean absolute percentage error:  0.31490652887182863
For ridge regression: 
r^2 value is  0.6703557685186001 
mean absolute percentage error:  0.3358030825798693
